---

## EXERCISE1: Burglary network

Implement the *Prior Sampling* algorithm to do approximate inference on last week's Burglary network.

<img src='https://drive.google.com/uc?id=11xqbchd4TCFSzdLVxNgr4SoeGfrSoqGO'>

Verify that the algorithm can correctly approximate the probability $P(j, m, a, \neg b, \neg e) = 0.00063$

Try different numbers of samples (e.g. $N = 10, 100, 1000, 10000$) and compare the results.

---

In [ ]:
import numpy as np
import random as rnd

t,f=0,1

P_B=np.array([0.001,0.999])
P_E=np.array([0.002,0.998])

PA_BE=np.array([[[0.95,0.94],[0.29,0.001]],[[0.05,0.06],[0.71,0.999]]])

PJ_A=np.array([[0.90,0.05],[0.10,0.95]])
PM_A=np.array([[0.70,0.01],[0.30,0.99]])

var=['B','E','A','J','M']

prd={'B':P_B, 'E':P_E,'A':PA_BE,'J':PJ_A,'M':PM_A}
par={'B':[],'E':[],'A':['B','E'],'J':['A'],'M':['A']}

val={'B':t, 'E':t,'A':t,'J':t,'M':t}

def samplegen(Pdist,Parents=[]):
  assert len(Parents)<len(Pdist.shape)
  #print(Parents)
  #print(tuple(Parents))
  #print(Pdist[t][tuple(Parents)])
  if rnd.random()<Pdist[t][tuple(Parents)]:#tuple converte lista in tupla
    return t
  return f

def parents(x):
  return [val[i] for i in par[x]]#prende il valore booleano dei genitori con cui si vuole campionare da val

#samplegen(prd['A'],parents('A'))

In [ ]:
event=[]

for i in range(1000000):
  for x in var:
    val[x]=samplegen(prd[x],parents(x))
  event.append(['f' if val[x] else 't' for x in var])

r=event.count(['f','f','t','t','t'])/len(event)

print(r)


0.000649


In [ ]:
!pip install pomegranate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pomegranate import *

In [ ]:
#bayesian networks model

burglary=DiscreteDistribution({'y':0.001, 'n':0.999})
earthquake=DiscreteDistribution({'y':0.002,'n':0.998})

alarm=ConditionalProbabilityTable(
    [['y','y','y',0.95],
     ['y','y','n',0.94],
     ['y','n','y',0.29],
     ['y','n','n',0.001],
     ['n','y','y',0.05],
     ['n','y','n',0.06],
     ['n','n','y',0.71],
     ['n','n','n',0.999]
     ],[burglary,earthquake]
)

marycalls=ConditionalProbabilityTable(
    [['y','y',0.70],
    ['y','n',0.30],
    ['n','y',0.01],
    ['n','n',0.99]],[alarm])

johncalls=ConditionalProbabilityTable(
    [['y','y',0.90],
    ['y','n',0.10],
    ['n','y',0.05],
    ['n','n',0.95]],[alarm])

s1=Node(burglary,name="Burglary")
s2=Node(earthquake, name="Earthquake")
s3=Node(alarm,name="Alarm")
s4=Node(johncalls,name="johnCalls")
s5=Node(marycalls,name="maryCalls")

model=BayesianNetwork("Alarm network")
model.add_states(s1,s2,s3,s4,s5)
model.add_edge(s1,s3)
model.add_edge(s2,s3)
model.add_edge(s3,s4)
model.add_edge(s3,s5)

model.bake()

NameError: ignored

---

## EXERCISE2: Pomegranate for Day 2

Use <tt>pomegranate</tt> to calculate the filtered probability of rain on Day 2 when we see an umbrella on Day 1 and Day 2.

What is the filtered probability of rain on Day 2 when we see <tt>not umbrella</tt> on Day 1?

How about if we just have no information about Umbrellas on Day 1 (i.e., only that rain on Day 2)?

In [ ]:
Rain0   = DiscreteDistribution({'y': 0.5, 'n': 0.5})

# Transition model
#
# Conditional distribution relating RainN and RainN+1. Notation for
# the conditional probability table is:
#
# [ 'RainN', 'RainN+1', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Note that we have to
# repeat the transition model for each pair of states
Rain1 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain0])

Rain2 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain1])

# Sensor model
#
# Conditional distribution relating Rain and Umbrella:
#
# [ 'Umbrella', 'Rain', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Values for Umbrella are 'y'es and 'n'o.
# Again we have to enter the table for each day.
Umbrella1 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain1])

Umbrella2 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain2])
#
# The whole network has five nodes:
s1 = Node(Rain0, name="Rain0")
s2 = Node(Rain1, name="Rain1")
s3 = Node(Umbrella1, name="Umbrella1")
s4 = Node(Rain2, name="Rain2")
s5 = Node(Umbrella2, name="Umbrella2")
# Create a network that includes nodes and edges between them:
model = BayesianNetwork("Umbrella Network")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s2)
model.add_edge(s2, s3)
model.add_edge(s2, s4)
model.add_edge(s4, s5)
# Fix the model structure
model.bake()


# Do not instantiate any of the variables:
scenario = [[None, None, 'y', None, 'y']]
# Run the model
results =  model.predict_proba(scenario)
# Ask for the probability of rain on Day 2:
print("Rain day 2 with u1 and u2", results[0][3].items())


# Do not instantiate any of the variables:
scenario = [[None, None, 'n', None, 'y']]
# Run the model
results =  model.predict_proba(scenario)
# Ask for the probability of rain on Day 2:
print("Rain day 2 with not u1", results[0][3].items())

scenario = [[None, None, None, None, 'y']]
# Run the model
results =  model.predict_proba(scenario)
# Ask for the probability of rain on Day 2:
print("Rain day 2 with u2",results[0][3].items())